In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('../data/Train_dataset.csv')
train.head(1)

,Registration Number,Annual Turnover,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,...,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [4]:
from kp_auto_ml import pre_processing as pp

In [5]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [6]:
train.head(1)

,registration_number,annual_turnover,cuisine,city,restaurant_location,opening_day_of_restaurant,facebook_popularity_quotient,endorsed_by,instagram_popularity_quotient,fire_audit,...,overall_restaurant_rating,live_music_rating,comedy_gigs_rating,value_deals_rating,live_sports_rating,ambience,lively,service,comfortablility,privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [7]:
label_encode:list[pp.PreLabelEncoderConfig] = []
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.city.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_location',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_location.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='endorsed_by',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.endorsed_by.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_theme',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_theme.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_type',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_type.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='cuisine',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cuisine.value_counts().keys())}))
    )

numeric_col_changer:list[pp.PreNumericColDataChangeConfig] = []
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='facebook_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='instagram_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='overall_restaurant_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='live_music_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='ambience',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='resturant_tier',data_type=int))

model_config = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column='annual_turnover'
)

In [8]:
pre_processed_df = pp.process(train,model_config)

Dropping columns whose values are nullable greater than 0.7
Removing unusable values like " " and "?"
Filling NA values with median and mode for numeria and non-numeric columns.
Performing IQR based outliers cleaning on target. annual_turnover
Changing datatype of numeric columns.
Performing label encoding on columns provided.
Encoding using dummies.


In [9]:
from kp_auto_ml import model_training_data_prep as dp
from kp_auto_ml import model_training_helper as mth
from kp_auto_ml import model_list_helper as mlh

In [10]:
data = dp.ModelTrainingData(pre_processed_df, dp.ScalerType.QUANTILE_TRANSFORMER, .85,use_pca=False,use_polynomials=False,use_feature_selection=False)
trainer = mth.ModelTrainer(data=data)
trainer.perform_operation_regression(exclude_models=[
    mlh.ModelAndParam.Linear_Regression
    # ,mlh.ModelAndParam.Ridge_Regression
    ,mlh.ModelAndParam.Lasso_Regression
    ,mlh.ModelAndParam.ElasticNet_Regression
    ,mlh.ModelAndParam.SVR_Regression
    ,mlh.ModelAndParam.DecisionTree_Regressor
    ,mlh.ModelAndParam.RandomForest_Regressor
    ,mlh.ModelAndParam.GradientBoosting_Regressor
    ,mlh.ModelAndParam.KNeighbors_Regressor

    ],permutate_n_less_column=0)
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

Total columns being used after all data transformations: 28
Ridge params: {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0], 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}
Total Permutations: 1


model_name    Ridge_Regression
score                 0.231142
RMSE           11233232.534757
Name: 0, dtype: object

In [94]:
from kp_auto_ml import neural_network_regression as nnr

In [95]:
trainer.perform_neural_network_regression()

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | -9.678e+1 | 5.51      | 335.3     | 0.4361    | 0.2308    | 43.63     | 1.298     | 1.045     | 0.426     | 31.48     | 0.3377    | 5.944     |
| 2         | -8.632e+1 | 2.14      | 265.0     | 0.6696    | 0.1864    | 41.94     | 1.932     | 1.237     | 0.08322   | 91.07     | 0.794     | 5.043     |
| 3         | -9.678e+1 | 4.69      | 264.7     | 0.9162    | 0.1493    | 43.12     | 1.487     | 1.564     | 0.6924    | 90.44     | 0.8239    | 2.537     |
| 4         | -9.678e+1 | 8.727     | 405.2     | 0.6846    | 0.1085    | 25.47     | 1.412     | 2.167     | 0.7813    | 44.51     | 0.02933   | 2.594     |


In [102]:
trainer.neural_network_best_model()

Model: "sequential_250"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1143 (Dense)          (None, 91)                2639      
                                                                 
 batch_normalization_138 (B  (None, 91)                364       
 atchNormalization)                                              
                                                                 
 dense_1144 (Dense)          (None, 91)                8372      
                                                                 
 dense_1145 (Dense)          (None, 91)                8372      
                                                                 
 dropout_153 (Dropout)       (None, 91)                0         
                                                                 
 dense_1146 (Dense)          (None, 91)                8372      
                                                    

In [103]:
trainer.performance_df

,model_name,score,RMSE
0,Ridge_Regression,0.231142,1.123323e+07
1,NeuralNetwork,-5.124911,3.179770e+07
2,NeuralNetwork,-5.099450,3.173155e+07


In [ ]:
test = pd.read_csv('../data/Test_dataset.csv')

In [ ]:
test.columns = test.columns.str.lower().str.replace(' ', '_')

In [ ]:
new_column_name = label_encode[2].column_name
test = test.rename(columns={'endoresed_by': new_column_name})

In [ ]:
label_encode[0] = pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(test.city.value_counts().keys())}))

In [ ]:
model_config_test = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column=''
)

In [ ]:
test = pp.process_test(test,model_config_test)

In [ ]:
test.city.value_counts()

In [ ]:
data.transform_test_data(test)